In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
def normalize(df):
    
    for column in ['x', 'y', 'z']:
        df[column] = (df[column] - df[column].mean()) / df[column].std()
    df = pd.DataFrame(df[['x', 'y', 'z']])
    return df

def segment(df, window_size, step_size):
    segments = []
    for start in range(0, len(df) - window_size + 1, step_size):
        end = start + window_size

        \
        segment = df.iloc[start:end]
        segments.append(segment)
    return segments

In [3]:
def segment_activities(window_size=32, step_size=16):
    # window_size = 160  # 2.5 seconds
    # step_size = 20  # 75% overlap

    activities_path = r'D:\PersonalFiles\MS_Analysis\Activity'
    activity_segments = {}

    # Iterate over each activity's folder
    for activity_name in os.listdir(activities_path):
        print(activity_name)
        activity_folder = os.path.join(activities_path, activity_name)
        if os.path.isdir(activity_folder):
            # Store segments for each activity
            activity_segments[activity_name] = []
            
            # Iterate over each CSV file within the activity's folder
            for filename in os.listdir(activity_folder):
                if filename.endswith('.csv'):
                    file_path = os.path.join(activity_folder, filename)
                    
                    # Read  CSV file
                    df = pd.read_csv(file_path)
                    
                    # Normalize data
                    df_normalized = normalize(df)
                    
                    # Segment the data using a rolling window
                    segments = segment(df_normalized, window_size, step_size)
                    
                    # Append the segments to the activity's list
                    activity_segments[activity_name].extend(segments)
    return activity_segments

In [4]:
def fit_model_SKLearn(activity_segments):
    X, y = [], []

    # Convert the segments into a suitable format for training
    for activity_name, segments in activity_segments.items():
        for segment in segments:
            # Assuming segment is a pandas DataFrame and flattening it to a 1D array
            feature_vector = segment.to_numpy().flatten()
            X.append(feature_vector)
            y.append(activity_name)
    X = np.array(X)
    y = np.array(y)
    print("segments in suitable format")

    # Encode the activity labels into integers
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    print("labels encoded")

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

    # Feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print("compiling model")
    # Define the SVM model
    model = svm.SVC(kernel='rbf', gamma='scale', C=1.0)

    print("training model")
    # Train the model
    model.fit(X_train_scaled, y_train)

    # Evaluate the model
    accuracy = model.score(X_test_scaled, y_test)
    print(f"Model evaluation complete. Accuracy: {accuracy}")

    return model


In [5]:
def fit_model_TensorFlow(activity_segments):
    X, y = [], []

    # Convert the segments into a suitable format for training
    for activity_name, segments in activity_segments.items():
        for segment in segments:
            feature_vector = segment.to_numpy().flatten()
            X.append(feature_vector)
            y.append(activity_name)
    print("segments in suitable format")

    # Encode the activity labels into integers
    X = np.array(X)
    y = np.array(y)

    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    print("labels encoded")

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Change labels to 1 and -1
    y_train = np.where(y_train > 0, 1, -1)
    y_test = np.where(y_test > 0, 1, -1)

    # Define the SVM model by Tensorflow
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(1, input_shape=(X_train_scaled.shape[1],))
    ])

    # Custom hinge loss
    def hinge_loss(y_true, y_pred):
        return tf.reduce_mean(tf.maximum(1 - y_true * y_pred, 0))

    model.compile(optimizer='sgd', loss=hinge_loss, metrics=['accuracy'])

    print("compiled model\nTraining Model")

    # Train the model
    model.fit(X_train_scaled, y_train, epochs=10, validation_split=0.2)

    # Evaluate the model
    model.evaluate(X_test_scaled, y_test)
    print(model.evaluate(X_test_scaled, y_test))
    
    return model

# 5 Second Window, 1 Second Overlap Data

Implementing SVM with Scikit-Learn

In [7]:
activity_segments = segment_activities(160, 36)
model = fit_model_SKLearn(activity_segments)

B1_T1_ (WALK)
B1_T2_ (WALK)
B1_TWT_A_ (WALK)
B1_TWT_B_ (WALK)
B2_T1_ (WALK)
B2_T2_ (WALK)
B2_TWT_A_ (WALK)
B2_TWT_B_ (WALK)
digit symbol task_ (SIT)
HR Recovery_ (STAND or SIT)
Montreal Cognitive Assessment_ (SIT)
Motor Behavioral
Motor Behavioral task
Motor behavioral task(1)
Motor Behavioral Task_ (All conditions seen here)
Naughton
Naughton Task
Naughton test
Naughton Test_ (WALK)
SOT_ (STAND)
TM comfortable speed
TM Comfortable speed(1)
TM Comfortable Speed_ (WALK)
TNT
trail making task_ (SIT)
Training ITWT - A
Training ITWT - B
Training ITWT -B
Training_ TWT_A
Training_ TWT_B
Training_TWT_A
Training_TWT_B
TWT_A training
TWT_A Training_ (WALK)
TWT_B training
TWT_B Training_ (WALK)
segments in suitable format
labels encoded
compiling model
training model


Implementing SVM with Tensorflow

In [6]:
activity_segments = segment_activities(160, 36)
model = fit_model_TensorFlow(activity_segments)

B1_T1_ (WALK)
B1_T2_ (WALK)
B1_TWT_A_ (WALK)
B1_TWT_B_ (WALK)
B2_T1_ (WALK)
B2_T2_ (WALK)
B2_TWT_A_ (WALK)
B2_TWT_B_ (WALK)
digit symbol task_ (SIT)
HR Recovery_ (STAND or SIT)
Montreal Cognitive Assessment_ (SIT)
Motor Behavioral
Motor Behavioral task
Motor behavioral task(1)
Motor Behavioral Task_ (All conditions seen here)
Naughton
Naughton Task
Naughton test
Naughton Test_ (WALK)
SOT_ (STAND)
TM comfortable speed
TM Comfortable speed(1)
TM Comfortable Speed_ (WALK)
TNT
trail making task_ (SIT)
Training ITWT - A
Training ITWT - B
Training ITWT -B
Training_ TWT_A
Training_ TWT_B
Training_TWT_A
Training_TWT_B
TWT_A training
TWT_A Training_ (WALK)
TWT_B training
TWT_B Training_ (WALK)
segments in suitable format
labels encoded


C:\Users\Lyu07\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


compiled model
Training Model
Epoch 1/10
3876/3876 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.8975 - loss: 0.1571 - val_accuracy: 0.9807 - val_loss: 0.0478
Epoch 2/10
3876/3876 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9803 - loss: 0.0483 - val_accuracy: 0.9815 - val_loss: 0.0447
Epoch 3/10
3876/3876 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9810 - loss: 0.0454 - val_accuracy: 0.9813 - val_loss: 0.0447
Epoch 4/10
3876/3876 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.9812 - loss: 0.0446 - val_accuracy: 0.9810 - val_loss: 0.0452
Epoch 5/10
3876/3876 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9809 - loss: 0.0449 - val_accuracy: 0.9815 - val_loss: 0.0445
Epoch 6/10
3876/3876 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.9813 - loss: 0.0439 - val_accuracy: 0.9817 - val_loss: 0.0424
Epoch 7/10
3876/3876 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9808 - loss: 0.0447 - val_accuracy: 0.9818 - val_loss: 0.0423
Epoch 8/10
3876/3876 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - ac

# FFT Data 5 Second Window, 1 Second Overlap

In [ ]:
import numpy as np
from numpy.fft import fft
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def apply_fft(segment):
    # Apply FFT and calculate the magnitude
    fft_values = fft(segment)
    magnitude = np.abs(fft_values)
    return magnitude

# Assuming 'activity_segments' is a dictionary with the structure mentioned earlier
X_fft = []  # FFT features
y = []  # Labels

for activity_name, segments in activity_segments.items():
    for segment in segments:
        # Assuming each segment is a pandas DataFrame with 'x', 'y', 'z' columns
        segment_fft = np.vstack([apply_fft(segment[col]) for col in ['x', 'y', 'z']]).T  # Apply FFT to each axis and transpose
        X_fft.append(segment_fft)
        y.append(activity_name)

X_fft = np.array(X_fft, dtype=object)
y = np.array(y)
print("segments in suitable format\nApplied FFT")

# Encode the activity labels into integers
le = LabelEncoder()
y_encoded = le.fit_transform(y)
print("encoded labels")
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_fft, y_encoded, test_size=0.2, random_state=42)

# Adjusting input shapes for LSTM based on FFT features
input_shape = (X_train[0].shape[0], X_train[0].shape[1])

# Define the LSTM model
model = Sequential([
    LSTM(128, input_shape=input_shape, return_sequences=True),
    LSTM(64),
    Dense(len(le.classes_), activation='softmax')
])
print("compiled model\nTraining Model")

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Since the datasets can have variable lengths due to FFT, use a generator or pad sequences for batching
# Example code for model training and evaluation is omitted due to complexity around handling variable input lengths



segments in suitable format
Applied FFT
encoded labels
compiled model
Training Model


In [ ]:
# Pad sequences to ensure uniform input size for LSTM
# Find the longest sequence
max_length = max(len(sample) for sample in X_fft)

# Pad all sequences to the length of the longest sequence
X_fft_padded = pad_sequences(X_fft, maxlen=max_length, dtype='float32', padding='post')
print("padded the data")
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_fft_padded, y_encoded, test_size=0.2, random_state=42)
print("training and testing data")
# Adjusting input shapes for LSTM based on FFT features
input_shape = (X_train.shape[1], X_train.shape[2])

# Define the LSTM model
model = Sequential([
    LSTM(128, input_shape=input_shape, return_sequences=True),
    LSTM(64),
    Dense(len(le.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print("compiled model")
# Train the model
print("training model")
model.fit(X_train, y_train, epochs=10, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

padded the data
training and testing data
compiled model
training model
Epoch 1/10
13943/13943 [==============================] - 1511s 108ms/step - loss: 1.8652 - accuracy: 0.3859 - val_loss: 1.6013 - val_accuracy: 0.4551
Epoch 2/10
13943/13943 [==============================] - 1526s 109ms/step - loss: 1.4807 - accuracy: 0.4891 - val_loss: 1.3956 - val_accuracy: 0.5121
Epoch 3/10
13943/13943 [==============================] - 1439s 103ms/step - loss: 1.2980 - accuracy: 0.5393 - val_loss: 1.2635 - val_accuracy: 0.5506
Epoch 4/10
13943/13943 [==============================] - 1598s 115ms/step - loss: 1.1838 - accuracy: 0.5724 - val_loss: 1.1821 - val_accuracy: 0.5701
Epoch 5/10
13943/13943 [==============================] - 1576s 113ms/step - loss: 1.1060 - accuracy: 0.5946 - val_loss: 1.1134 - val_accuracy: 0.5917
Epoch 6/10
13943/13943 [==============================] - 1494s 107ms/step - loss: 1.0490 - accuracy: 0.6104 - val_loss: 1.0971 - val_accuracy: 0.5957
Epoch 7/10
13943/13943

# 3 Second Window 3/2 Second Overlap

In [7]:
activity_segments = segment_activities(96, 12)
model = fit_model_TensorFlow(activity_segments)

B1_T1_ (WALK)
B1_T2_ (WALK)
B1_TWT_A_ (WALK)
B1_TWT_B_ (WALK)
B2_T1_ (WALK)
B2_T2_ (WALK)
B2_TWT_A_ (WALK)
B2_TWT_B_ (WALK)
digit symbol task_ (SIT)
HR Recovery_ (STAND or SIT)
Montreal Cognitive Assessment_ (SIT)
Motor Behavioral
Motor Behavioral task
Motor behavioral task(1)
Motor Behavioral Task_ (All conditions seen here)
Naughton
Naughton Task
Naughton test
Naughton Test_ (WALK)
SOT_ (STAND)
TM comfortable speed
TM Comfortable speed(1)
TM Comfortable Speed_ (WALK)
TNT
trail making task_ (SIT)
Training ITWT - A
Training ITWT - B
Training ITWT -B
Training_ TWT_A
Training_ TWT_B
Training_TWT_A
Training_TWT_B
TWT_A training
TWT_A Training_ (WALK)
TWT_B training
TWT_B Training_ (WALK)
segments in suitable format
labels encoded


C:\Users\Lyu07\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


compiled model
Training Model
Epoch 1/10
11668/11668 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step - accuracy: 0.9489 - loss: 0.0887 - val_accuracy: 0.9810 - val_loss: 0.0430
Epoch 2/10
11668/11668 ━━━━━━━━━━━━━━━━━━━━ 5s 436us/step - accuracy: 0.9810 - loss: 0.0428 - val_accuracy: 0.9810 - val_loss: 0.0418
Epoch 3/10
11668/11668 ━━━━━━━━━━━━━━━━━━━━ 5s 433us/step - accuracy: 0.9812 - loss: 0.0419 - val_accuracy: 0.9809 - val_loss: 0.0429
Epoch 4/10
11668/11668 ━━━━━━━━━━━━━━━━━━━━ 5s 424us/step - accuracy: 0.9812 - loss: 0.0421 - val_accuracy: 0.9810 - val_loss: 0.0426
Epoch 5/10
11668/11668 ━━━━━━━━━━━━━━━━━━━━ 5s 427us/step - accuracy: 0.9808 - loss: 0.0429 - val_accuracy: 0.9811 - val_loss: 0.0415
Epoch 6/10
11668/11668 ━━━━━━━━━━━━━━━━━━━━ 5s 438us/step - accuracy: 0.9815 - loss: 0.0412 - val_accuracy: 0.9802 - val_loss: 0.0475
Epoch 7/10
11668/11668 ━━━━━━━━━━━━━━━━━━━━ 5s 432us/step - accuracy: 0.9816 - loss: 0.0413 - val_accuracy: 0.9801 - val_loss: 0.0471
Epoch 8/10
11668/11668 ━━━━━━━━━━

# 1 Second Window 1/2 Second Overlap

In [8]:
activity_segments = segment_activities()
model = fit_model_TensorFlow(activity_segments)

B1_T1_ (WALK)
B1_T2_ (WALK)
B1_TWT_A_ (WALK)
B1_TWT_B_ (WALK)
B2_T1_ (WALK)
B2_T2_ (WALK)
B2_TWT_A_ (WALK)
B2_TWT_B_ (WALK)
digit symbol task_ (SIT)
HR Recovery_ (STAND or SIT)
Montreal Cognitive Assessment_ (SIT)
Motor Behavioral
Motor Behavioral task
Motor behavioral task(1)
Motor Behavioral Task_ (All conditions seen here)
Naughton
Naughton Task
Naughton test
Naughton Test_ (WALK)
SOT_ (STAND)
TM comfortable speed
TM Comfortable speed(1)
TM Comfortable Speed_ (WALK)
TNT
trail making task_ (SIT)
Training ITWT - A
Training ITWT - B
Training ITWT -B
Training_ TWT_A
Training_ TWT_B
Training_TWT_A
Training_TWT_B
TWT_A training
TWT_A Training_ (WALK)
TWT_B training
TWT_B Training_ (WALK)
segments in suitable format
labels encoded
compiled model
Training Model
Epoch 1/10


C:\Users\Lyu07\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8795/8795 ━━━━━━━━━━━━━━━━━━━━ 4s 439us/step - accuracy: 0.9446 - loss: 0.0933 - val_accuracy: 0.9803 - val_loss: 0.0422
Epoch 2/10
8795/8795 ━━━━━━━━━━━━━━━━━━━━ 4s 423us/step - accuracy: 0.9813 - loss: 0.0398 - val_accuracy: 0.9804 - val_loss: 0.0406
Epoch 3/10
8795/8795 ━━━━━━━━━━━━━━━━━━━━ 4s 417us/step - accuracy: 0.9812 - loss: 0.0395 - val_accuracy: 0.9804 - val_loss: 0.0404
Epoch 4/10
8795/8795 ━━━━━━━━━━━━━━━━━━━━ 4s 421us/step - accuracy: 0.9814 - loss: 0.0391 - val_accuracy: 0.9804 - val_loss: 0.0409
Epoch 5/10
8795/8795 ━━━━━━━━━━━━━━━━━━━━ 4s 446us/step - accuracy: 0.9809 - loss: 0.0401 - val_accuracy: 0.9804 - val_loss: 0.0405
Epoch 6/10
8795/8795 ━━━━━━━━━━━━━━━━━━━━ 4s 421us/step - accuracy: 0.9817 - loss: 0.0383 - val_accuracy: 0.9804 - val_loss: 0.0405
Epoch 7/10
8795/8795 ━━━━━━━━━━━━━━━━━━━━ 4s 433us/step - accuracy: 0.9816 - loss: 0.0385 - val_accuracy: 0.9804 - val_loss: 0.0405
Epoch 8/10
8795/8795 ━━━━━━━━━━━━━━━━━━━━ 4s 426us/step - accuracy: 0.9815 - loss: 0.03